In [13]:
import time
import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict
import pyomo.environ as pyo

from gep_main import run_model, prep_data
from gep_problem import GEPProblem

import torch

DTYPE = torch.float64

torch.set_default_dtype(DTYPE)

# DEVICE = (
#     torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# )

DEVICE = "cpu"


In [14]:

CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"

HIGHS  = "HiGHS"
GUROBI = "Gurobi"

SAMPLE_DURATION = 12 # 12 hours

## Step 0: Activate environment - ensure consistency accross computers
# print("Reading the data")
# print("Activating the environment")


In [15]:

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

print("Initializing the solver")
optimizer_name = data["optimizer_config"]["solver"]

# Determine the optimizer
if optimizer_name == HIGHS:
    raise NotImplementedError(f"{optimizer_name}: Not implemented")
elif optimizer_name == GUROBI:
    
    print(f"Using {GUROBI}")
    optimizer = "gurobi_direct"
else:
    raise ValueError(f"{optimizer_name}: Not implemented")

Parsing the config file
Initializing the solver
Using Gurobi


In [16]:
for i, experiment_instance in enumerate(experiment["experiments"]):
    # Setup output dataframe
    df_res = pd.DataFrame(columns=["setup_time", "presolve_time", "barrier_time", "crossover_time", "restore_time", "objective_value"])

    T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap = prep_data(experiment_instance)

    T_ranges = [range(i, i + SAMPLE_DURATION, 1) for i in range(1, len(T), SAMPLE_DURATION)]
    objective_values = []
    times = []
    for t in T_ranges[:1]:
        # Run one experiment for j repeats
        model, solver, time_taken = run_model(experiment_instance, t, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap)
        
        # print("Print values for all variables")
        # for v in model.component_data_objects(pyo.Var):
        #     print(str(v), v.value)

        objective_values.append(model.obj())
        times.append(time_taken)

    # Write the number to the file
    # file_path = os.path.join("outputs/Gurobi", f"samplesize_{str(SAMPLE_DURATION)}_node_{len(N)}_gen_{len(G)}_lines_{len(L)}.txt")
    # with open(file_path, "w") as file:
    #     file.write(f"N: {str(N)}\n")
    #     file.write(f"G: {str(G)}\n")
    #     file.write(f"L: {str(L)}\n")
    #     file.write(f"Obj value mean: {str(np.mean(objective_values))}\n")
    #     file.write(f"Time taken mean: {str(np.mean(times))}\n")

Wrangling the input data
Populating the model
Adding model variables
Formulating the objective
Adding model constraints
Solving the optimization problem
Set parameter OutputFlag to value 1
Set parameter Method to value 2
Set parameter OutputFlag to value 1
Set parameter LogFile to value "outputs/Gurobi/output.txt"
Set parameter Crossover to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
Method  2
Crossover  0

Optimize a model with 140 rows, 113 columns and 467 nonzeros
Model fingerprint: 0x93c00021
Coefficient statistics:
  Matrix range     [7e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+03, 3e+04]
  RHS range        [4e+03, 3e+04]
Presolve removed 126 rows and 104 columns
Presolve time: 0.00s
Presolved: 14 rows, 9 columns, 42 nonzeros
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     :

In [17]:
for var in model.component_objects(pyo.Var, active=True):
        for index in var:
            print(f"{var.name}[{index}] = {var[index].value}")


vInvCost[None] = 30091210.065572247
vOpeCost[None] = 788556822.2172318
vGenInv[('BEL', 'SunPV')] = 3188.980650317394
vGenInv[('GER', 'SunPV')] = 15130.885774262544
vGenInv[('NED', 'SunPV')] = 6756.141963396936
vGenProd[('BEL', 'SunPV', 1)] = 0.0
vGenProd[('BEL', 'SunPV', 2)] = 0.0
vGenProd[('BEL', 'SunPV', 3)] = 0.0
vGenProd[('BEL', 'SunPV', 4)] = 0.0
vGenProd[('BEL', 'SunPV', 5)] = 0.0
vGenProd[('BEL', 'SunPV', 6)] = 0.0
vGenProd[('BEL', 'SunPV', 7)] = 0.0
vGenProd[('BEL', 'SunPV', 8)] = 0.0
vGenProd[('BEL', 'SunPV', 9)] = 3986.225812896743
vGenProd[('BEL', 'SunPV', 10)] = 3646.9423378464453
vGenProd[('BEL', 'SunPV', 11)] = 3942.8316931533423
vGenProd[('BEL', 'SunPV', 12)] = 4305.202009367987
vGenProd[('GER', 'SunPV', 1)] = 0.0
vGenProd[('GER', 'SunPV', 2)] = 0.0
vGenProd[('GER', 'SunPV', 3)] = 0.0
vGenProd[('GER', 'SunPV', 4)] = 0.0
vGenProd[('GER', 'SunPV', 5)] = 0.0
vGenProd[('GER', 'SunPV', 6)] = 0.0
vGenProd[('GER', 'SunPV', 7)] = 0.0
vGenProd[('GER', 'SunPV', 8)] = 0.0
vGenProd[

In [18]:

print("Constraint values:")
for con in model.component_objects(pyo.Constraint, active=True):
    for index in con:
        constraint_expr = con[index].body
        constraint_lb = con[index].lower
        constraint_ub = con[index].upper
        constraint_value = pyo.value(constraint_expr)  # Evaluate the constraint body
        print(f"{con.name}[{index}]:")
        print(f"  Expression Value: {constraint_value}")
        if constraint_lb is not None:
            print(f"  Lower Bound: {constraint_lb}, Feasibility: {constraint_value >= constraint_lb}")
        if constraint_ub is not None:
            print(f"  Upper Bound: {constraint_ub}, Feasibility: {constraint_value <= constraint_ub}")


Constraint values:
eInvCost[None]:
  Expression Value: 0.0
  Lower Bound: 0.0, Feasibility: True
  Upper Bound: 0.0, Feasibility: True
eOpeCost[None]:
  Expression Value: 0.0
  Lower Bound: 0.0, Feasibility: True
  Upper Bound: 0.0, Feasibility: True
eNodeBal[('BEL', 1)]:
  Expression Value: 5336.38128679075
  Lower Bound: 5336.38128679075, Feasibility: True
  Upper Bound: 5336.38128679075, Feasibility: True
eNodeBal[('BEL', 2)]:
  Expression Value: 4884.63342930747
  Lower Bound: 4884.63342930747, Feasibility: True
  Upper Bound: 4884.63342930747, Feasibility: True
eNodeBal[('BEL', 3)]:
  Expression Value: 4489.7215278633
  Lower Bound: 4489.7215278633, Feasibility: True
  Upper Bound: 4489.7215278633, Feasibility: True
eNodeBal[('BEL', 4)]:
  Expression Value: 4159.48502467053
  Lower Bound: 4159.48502467053, Feasibility: True
  Upper Bound: 4159.48502467053, Feasibility: True
eNodeBal[('BEL', 5)]:
  Expression Value: 3890.98412889956
  Lower Bound: 3890.98412889956, Feasibility: Tru

In [19]:
def get_variable_values_as_list(var):
    # Check if the variable is indexed
    if var.is_indexed():
        return [var[idx].value for idx in var]
    else:
        # For scalar variables
        return [var.value]

# Function to get constraint values as a list
def get_constraint_values_as_list(constraint):
    constraint_values = []
    for idx in constraint:
        # Evaluate the constraint body
        constraint_expr_value = pyo.value(constraint[idx].body)
        constraint_values.append(constraint_expr_value)
    return constraint_values

def get_variable_bounds(var):
    lower_bounds = []
    upper_bounds = []
    
    # Check if the variable is indexed
    if var.is_indexed():
        # Iterate through the indices of the variable
        for idx in var:
            lower_bounds.append(var[idx].lb)  # Get lower bound for each index
            upper_bounds.append(var[idx].ub)  # Get upper bound for each index
    else:
        # For scalar variables
        lower_bounds.append(var.lb)  # Get lower bound of scalar variable
        upper_bounds.append(var.ub)  # Get upper bound of scalar variable

    return lower_bounds, upper_bounds


# !!!! 
### When using MPS, calculations are done in float32. This introduces errors of the order e-3. 

In [20]:
#T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap
data = GEPProblem(T, G, L, N, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap)

# X contains [pGenAva, pDemand]
X = data.trainX[:1]
# Y contains [*vGenProd, *vLineFlow, *vLossLoad, *vGenInv]

# Y = [model.vGenProd, model.vLineFlow, model.vLossLoad, model.vGenInv]
Y = torch.tensor([[*get_variable_values_as_list(model.vGenProd), \
     *get_variable_values_as_list(model.vLineFlow),
     *get_variable_values_as_list(model.vLossLoad), 
     *get_variable_values_as_list(model.vGenInv)]], device=DEVICE)

g = data.g(X, Y).squeeze(0).tolist()

Size of train set: 584
Size of val set: 73
Size of mu: 285
Size of lambda: 36
Number of variables (size of y): 111
Number of inputs (size of X): 72


In [21]:
print("### Testing constistency of g_x(y) ###")

# g returns: [eMaxProd, eLineFlow, eRampingDown, eRampingUp, eGenProdPositive, eMissedDemandPositive, eNumPowerGenerationUnitsPositive]
print("Testing consistency of eMaxProd")
eMaxProd_idx = len(G) * 12
eMaxProd_PDL = g[:eMaxProd_idx]
eMaxProd_LP = get_constraint_values_as_list(model.eMaxProd)
assert(eMaxProd_PDL == eMaxProd_LP)

print("Testing consistency of eLineFlow")
# Compute indices for slicing
eLineFlow_idx_lb = eMaxProd_idx + len(L) * 12
eLineFlow_idx_ub = eLineFlow_idx_lb + len(L) * 12

# Extract values using the indices
eLineFlow_PDL_lb = np.array(g[eMaxProd_idx:eLineFlow_idx_lb], dtype=np.float64)
eLineFlow_PDL_ub = np.array(g[eLineFlow_idx_lb:eLineFlow_idx_ub], dtype=np.float64)

# Get bounds and variable values as numpy arrays in float64
eLineFlow_LP_lb, eLineFlow_LP_ub = (
    np.array(get_variable_bounds(model.vLineFlow)[0], dtype=np.float64),  # Lower bounds
    np.array(get_variable_bounds(model.vLineFlow)[1], dtype=np.float64),  # Upper bounds
)
vLineFlow = np.array(get_variable_values_as_list(model.vLineFlow), dtype=np.float64)

# Assertions tolerate errors due to floating point precision
assert np.allclose(eLineFlow_PDL_lb + vLineFlow, eLineFlow_LP_lb, atol=1e-10), (
    f"Assertion failed: eLineFlow_PDL_lb + vLineFlow != eLineFlow_LP_lb\n"
    f"  eLineFlow_PDL_lb + vLineFlow = {(eLineFlow_PDL_lb + vLineFlow).tolist()}\n"
    f"  eLineFlow_LP_lb = {eLineFlow_LP_lb.tolist()}"
)
assert np.allclose(-1 * (eLineFlow_PDL_ub - vLineFlow), eLineFlow_LP_ub, atol=1e-10), (
    f"Assertion failed: -1 * (eLineFlow_PDL_ub - vLineFlow) != eLineFlow_LP_ub\n"
    f"  -1 * (eLineFlow_PDL_ub - vLineFlow) = {(-1 * (eLineFlow_PDL_ub - vLineFlow)).tolist()}\n"
    f"  eLineFlow_LP_ub = {eLineFlow_LP_ub.tolist()}"
)

print("Testing consistency of eRampingDown")
eRampingDown_idx = eLineFlow_idx_ub + len(G) * (12-1)
eRampingDown_PDL = g[eLineFlow_idx_ub:eRampingDown_idx]
eRampingDown_LP = get_constraint_values_as_list(model.eRampingDown)

assert np.allclose(np.array(eRampingDown_PDL), np.array(eRampingDown_LP), atol=1e-10)

print("Testing consistency of eRampingUp")
eRampingUp_idx = eRampingDown_idx + len(G) * (12-1)
eRampingUp_PDL = g[eRampingDown_idx:eRampingUp_idx]
eRampingUp_LP = get_constraint_values_as_list(model.eRampingUp)

assert np.allclose(np.array(eRampingUp_PDL), np.array(eRampingUp_LP), atol=1e-10)

print("Testing consistency of eGenProdPositive")
eGenProdPositive_idx = eRampingUp_idx + len(G) * 12
eGenProdPositive_PDL = g[eRampingUp_idx:eGenProdPositive_idx]

vGenProd = get_variable_values_as_list(model.vGenProd)

assert np.allclose(np.array(eGenProdPositive_PDL), -1*np.array(vGenProd), atol=1e-10)

print("Testing consistency of eMissedDemandPositive")
eMissedDemandPositive_idx = eGenProdPositive_idx + len(N) * 12
eMissedDemandPositive_PDL = g[eGenProdPositive_idx:eMissedDemandPositive_idx]

vLossLoad = get_variable_values_as_list(model.vLossLoad)

assert np.allclose(np.array(eMissedDemandPositive_PDL), -1*np.array(vLossLoad), atol=1e-10)

print("Testing consistency of eMissedDemandLeqDemand")
eMissedDemandLeqDemand_idx = eMissedDemandPositive_idx + len(N) * 12
eMissedDemandLeqDemand_PDL = g[eMissedDemandPositive_idx:eMissedDemandLeqDemand_idx]

vLossLoad = get_variable_values_as_list(model.vLossLoad)
pDemand = data.pDemand_tensor[:, :12].flatten().tolist()

assert np.allclose(np.array(eMissedDemandLeqDemand_PDL), np.array(vLossLoad) - np.array(pDemand), atol=1e-10)

print("Testing consistency of eNumPowerGenerationUnitsPositive")
eNumPowerGenerationUnitsPositive_idx = eMissedDemandLeqDemand_idx + len(G)
eNumPowerGenerationUnitsPositive_PDL = g[eMissedDemandLeqDemand_idx:eNumPowerGenerationUnitsPositive_idx]


vGenInv = get_variable_values_as_list(model.vGenInv)

assert np.allclose(np.array(eNumPowerGenerationUnitsPositive_PDL), -1*np.array(vGenInv), atol=1e-10)







### Testing constistency of g_x(y) ###
Testing consistency of eMaxProd
Testing consistency of eLineFlow
Testing consistency of eRampingDown
Testing consistency of eRampingUp
Testing consistency of eGenProdPositive
Testing consistency of eMissedDemandPositive
Testing consistency of eMissedDemandLeqDemand
Testing consistency of eNumPowerGenerationUnitsPositive


In [22]:
h = data.h(X, Y).squeeze(0).tolist()

In [23]:
print("### Testing consistency of h_x(y) ###")
# h returns eNodeBal

print("Testing consistency of eNodeBal")
eNodeBal_PDL = h
eNodeBal_LP = get_constraint_values_as_list(model.eNodeBal)

assert np.allclose(np.array(eNodeBal_PDL), np.array(eNodeBal_LP) - np.array(pDemand), atol=1e-10)


### Testing consistency of h_x(y) ###
Testing consistency of eNodeBal


In [ ]:
print("### Testing consistency of f_x(y)")

obj_LP = model.obj()

obj_PDL = data.obj_fn(Y)

assert obj_LP == obj_PDL

print(obj_LP)
print(obj_PDL)


### Testing consistency of f_x(y)
torch.Size([1, 111])
818648032.282804
tensor([8.1865e+08])
